### Save and Retrive Data from Redis
- https://redis.com/

In [1]:
import numpy as np
import pandas as pd
import redis

In [2]:
hostname = "redis-12004.c13.us-east-1-3.ec2.cloud.redislabs.com"
port = 12004
password = "5w8GS5MT0PujpWOk8kkKL06Av1gapTTq"

r = redis.StrictRedis(host=hostname,
                port=port,
                password=password)

In [3]:
r.ping()

True

In [6]:
#r.keys()

#### Load the data (numpy zip)

In [4]:
np_file = np.load('dataframe_students_teacher.npz',allow_pickle=True)
x_values = np_file['arr_0']
col_names = np_file['arr_1']

df = pd.DataFrame(x_values,columns=col_names)
df

,name,role,facial_features
0,Angelina Jolie,Student,"[0.19881923, 1.4748836, -0.097958915, -0.88050..."
1,Barack Obama,Teacher,"[-0.22177818, 1.5762168, -0.503078, 0.13601258..."
2,Chris Evans,Student,"[0.35691038, -0.4149104, 0.039218407, -0.15436..."
3,Morgan Freeman,Teacher,"[0.7627893, 0.89061487, 0.46873552, 0.2589808,..."
4,Scarlett Johansson,Student,"[-0.13535991, -0.28213844, -0.42700863, -0.282..."


In [5]:
df['name_role'] = df['name']+'@'+df['role']
df

,name,role,facial_features,name_role
0,Angelina Jolie,Student,"[0.19881923, 1.4748836, -0.097958915, -0.88050...",Angelina Jolie@Student
1,Barack Obama,Teacher,"[-0.22177818, 1.5762168, -0.503078, 0.13601258...",Barack Obama@Teacher
2,Chris Evans,Student,"[0.35691038, -0.4149104, 0.039218407, -0.15436...",Chris Evans@Student
3,Morgan Freeman,Teacher,"[0.7627893, 0.89061487, 0.46873552, 0.2589808,...",Morgan Freeman@Teacher
4,Scarlett Johansson,Student,"[-0.13535991, -0.28213844, -0.42700863, -0.282...",Scarlett Johansson@Student


In [6]:
records = df[['name_role','facial_features']].to_dict(orient='records')
len(records)

5

## Save data in Redis
- redis hashes
- keyname = 'academy:register'

In [7]:
for record in records:
    name_role = record['name_role']
    vector = record['facial_features']
    
    # convert numpy array into bytes
    vector_bytes = vector.tobytes()
    
    # save data into redis cloud
    r.hset(name='academy:register',key=name_role,value=vector_bytes)

In [8]:
vector.dtype

dtype('float32')

### Retrive Records from Redis

In [9]:
retrive_dict = r.hgetall(name='academy:register')
retrive_series = pd.Series(retrive_dict)
retrive_series

b'Chris Evans@Student'           b'\xf5\xbc\xb6>#o\xd4\xbe{\xa3 =\xba\x11\x1e\x...
b'Angelina Jolie@Student'        b'E\x97K>\xfc\xc8\xbc?\xaf\x9e\xc8\xbd\xd2ha\x...
b'Morgan Freeman@Teacher'        b')FC?V\xffc?\x1a\xfe\xef>"\x99\x84>\xee\xfcb\...
b'Scarlett Johansson@Student'    b'\xca\x9b\n\xbest\x90\xbe\xe0\xa0\xda\xbe\x0c...
b'Barack Obama@Teacher'          b'\xd2\x19c\xbey\xc1\xc9?\xb8\xc9\x00\xbf\xe2F...
dtype: object

In [10]:
#np.frombuffer(retrive_series[b'Morgan Freeman@Teacher'],dtype=np.float32).shape
retrive_series = retrive_series.apply(lambda x: np.frombuffer(x,dtype=np.float32))
index = retrive_series.index
index = list(map(lambda x: x.decode(), index))

In [11]:
retrive_series.index = index

In [12]:
retrive_df = retrive_series.to_frame().reset_index()
retrive_df.columns = ['name_role','facial_features']
retrive_df

,name_role,facial_features
0,Chris Evans@Student,"[0.35691038, -0.4149104, 0.039218407, -0.15436..."
1,Angelina Jolie@Student,"[0.19881923, 1.4748836, -0.097958915, -0.88050..."
2,Morgan Freeman@Teacher,"[0.7627893, 0.89061487, 0.46873552, 0.2589808,..."
3,Scarlett Johansson@Student,"[-0.13535991, -0.28213844, -0.42700863, -0.282..."
4,Barack Obama@Teacher,"[-0.22177818, 1.5762168, -0.503078, 0.13601258..."


### Registration Form

In [13]:
import cv2
from insightface.app import FaceAnalysis

In [14]:
# configure face analysis
faceapp = FaceAnalysis(name='buffalo_l',
                       root='insightface_model',
                       providers=['CPUExecutionProvider'])

faceapp.prepare(ctx_id=0, det_size=(640,640), det_thresh=0.5)
# warning: don't set det_thresh < 0.3

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


##### Step-1: Collect Person and Role

In [15]:
# python input
person_name = input('Enter your name: ')

trials = 3
for i in range(trials):
    role = input("""
    Please choose
    1. Student
    2. Teacher

    Enter number either 1 or 2
    """)
    
    if role in ('1','2'):
        if role == '1':
            role = 'Student'
        else:
            role = 'Teacher'
        
        break
    else:
        print('Invalid entry try again')
        if i == 3:
            print('Exceeds maximum trails')
            
key = person_name +'@'+ role
print('Your name = ', person_name)
print('Your role =', role)
print('Key =',key)

Enter your name: onyeka

    Please choose
    1. Student
    2. Teacher

    Enter number either 1 or 2
    1
Your name =  onyeka
Your role = Student
Key = onyeka@Student


##### Step-2: Collect Samples

In [16]:
cap = cv2.VideoCapture(0) # 0 default camera, 1 external camera
face_embeddings = []
sample = 0
while True:
    ret, frame = cap.read()
    if ret == False:
        print('Unable to read camera')
        break
    
    # get results from insightface model
    results = faceapp.get(frame,max_num=1)
    for res in results:
        sample += 1
        x1, y1, x2, y2 = res['bbox'].astype(int)
        cv2.rectangle(frame, (x1,y1),(x2,y2),(0,255,0),1)
        
        # facial features
        embeddings = res['embedding']
        face_embeddings.append(embeddings)
        
    if sample >= 200: 
        break
    
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) == ord('q'): # this is trigger only when I press letter q (lowercase q) in my keyboard
        break
        
cap.release()
cv2.destroyAllWindows()

c:\Users\onyek\Documents\PROJECTS\face_rec_attendance_system\.venv\lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


In [17]:
len(face_embeddings)

200

##### Step-3: Optimize Data & Save in Redis Database

In [18]:
x_mean = np.asarray(face_embeddings).mean(axis=0)
x_mean.shape

(512,)

In [82]:
# convert into bytes
x_mean_bytes = x_mean.tobytes()

In [83]:
# save key and facial bytes into redis database
r.hset(name='academy:register',key=key,value=x_mean_bytes)

1